In [51]:
import numpy as np
import pandas as pd

In [52]:
cases = pd.read_csv("C:/Users/shrof/Downloads/data100k.csv")

In [53]:
cases.head(3).T

,0,1,2
person_id,102090000000110,343221000000125,343221000000125
HearingDate,2019-02-28,2009-12-07,2011-01-20
CodeSection,A.46.2-862,B.46.2-301,A.46.2-707
codesection,covered elsewhere,covered elsewhere,covered elsewhere
ChargeType,Misdemeanor,Misdemeanor,Misdemeanor
chargetype,Misdemeanor,Misdemeanor,Misdemeanor
Class,1,1,3
DispositionCode,Guilty,Guilty,Guilty
disposition,Conviction,Conviction,Conviction
Plea,NaN,NaN,NaN


In [54]:
## What code sections are most frequent? 

In [55]:
codesections = cases['CodeSection'].value_counts().reset_index()
codesections.head(15)

,CodeSection,count
0,A.46.2-862,26379
1,B.46.2-301,25967
2,46.2-300,17934
3,C.46.2-862,11728
4,18.2-250.1,10573
5,A.18.2-266,8568
6,18.2-95,7561
7,18.2-250,6949
8,18.2-57,6699
9,A.46.2-852,6667


### Which codes lead to most convictions? 

In [61]:
cases['DispositionCode'].value_counts()
cases['conviction'] = [x in ['Guilty', 'Guilty In Abstentia'] for x in cases['DispositionCode']]

In [65]:
convict_rate = cases.groupby('CodeSection').agg({'conviction':['mean','count']}).reset_index()
convict_rate.columns = ['CodeSection', 'conviction_rate', 'count']
convict_rate = convict_rate.query("count >= 30")
convict_rate.sort_values('conviction_rate', ascending=False)

,CodeSection,conviction_rate,count
1633,21-336,0.960000,50
737,18.2-195(1)(A),0.926829,41
1538,21-1/46.2-301,0.913043,46
4111,G.18.2-266,0.906977,43
4012,B.46.2-357,0.904891,736
...,...,...,...
2690,41.1-2-2,0.015504,129
253,13-60,0.014286,70
1433,19.2-100,0.000000,238
140,11.1-2,0.000000,38


In [59]:
cases.query("CodeSection == '23-55'")['fips']

8585      810
17940     810
34148     810
34168     810
34169     810
46925     810
68396     810
120838    810
120844    810
120862    810
120864    810
121263    810
132073    810
153191    810
153206    810
153208    810
153210    810
153222    810
155161    810
157747    810
159548    810
163720    810
173408    810
192200    810
196647    810
200799    810
200828    810
200922    810
230213    810
240905    810
240909    810
240912    810
240914    810
240916    810
240917    810
240918    810
240919    810
240920    810
240922    810
240923    810
240926    810
240927    810
240945    810
240946    810
240948    810
249880    810
251381    810
257688    810
257697    810
257699    810
257739    810
257747    810
266159    810
291045    810
291048    810
Name: fips, dtype: int64

### Most racial disparities

In [66]:
# filter out ones with small counts
cases['Race'].unique()

array(['Black(Non-Hispanic)', 'Hispanic', 'White Caucasian(Non-Hispanic)',
       'MISSING', 'Asian Or Pacific Islander', 'Black (Non-Hispanic)',
       'White Caucasian (Non-Hispanic)',
       'Other(Includes Not Applicable.. Unknown)',
       'Other (Includes Not Applicable.. Unknown)', 'Black', 'White',
       'Unknown (Includes Not Applicable.. Unknown)', 'American Indian',
       'Unknown', 'Asian or Pacific Islander',
       'American Indian Or Alaskan Native'], dtype=object)

In [67]:
replace_map = {'Black(Non-Hispanic)':'Black',
'Hispanic':'Hispanic', 
'White Caucasian(Non-Hispanic)':'White',
'MISSING':'Missing/Other', 
'Asian Or Pacific Islander':'Asian or Pacific Islander',
'Black (Non-Hispanic)':'Black',
'White Caucasian (Non-Hispanic)':'White',
'Other(Includes Not Applicable.. Unknown)':'Missing/Other',
'Other (Includes Not Applicable.. Unknown)':'Missing/Other',
'Black':'Black', 
'White':'White',
'Unknown (Includes Not Applicable.. Unknown)':'Missing/Other',
'American Indian':'American Indian or Alaskan Native',
'Unknown':'', 
'Asian or Pacific Islander':'Asian or Pacific Islander',
'American Indian Or Alaskan Native':'American Indian or Alaskan Native'}

cases['Race'] = cases['Race'].replace(replace_map)
cases['Race'].value_counts()

Race
White                                159627
Black                                115627
Hispanic                               9319
Missing/Other                          5874
Asian or Pacific Islander              2794
American Indian or Alaskan Native       303
                                         54
Name: count, dtype: int64

In [69]:
# I choose to analyze only the convictions

In [70]:
cases_convict = cases.query("conviction == True")
cases_convict_race = cases_convict.groupby(['CodeSection', 'Race']).size().reset_index()
cases_convict_race = cases_convict_race.rename({0:'count'}, axis=1)
cases_convict_race

,CodeSection,Race,count
0,01-2007,White,1
1,1,Black,1
2,1,White,1
3,1-12,Black,24
4,1-12,White,5
...,...,...,...
4261,Z.18.2-91,Hispanic,2
4262,Z.18.2-91,White,123
4263,Z.18.2-91; 26,Black,1
4264,Z.18.2-95,Black,2


In [71]:
cases_reshape = cases_convict_race.pivot_table(index = 'CodeSection', columns = 'Race', 
                                               values = 'count', fill_value=0).reset_index()
cases_reshape

Race,CodeSection,,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White
0,01-2007,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1-12,0.0,0.0,0.0,24.0,0.0,0.0,5.0
3,1-200,0.0,0.0,0.0,7.0,0.0,0.0,3.0
4,1.21,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2672,Z.18.2-90,0.0,0.0,0.0,4.0,0.0,1.0,3.0
2673,Z.18.2-91,0.0,0.0,2.0,95.0,2.0,0.0,123.0
2674,Z.18.2-91; 26,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2675,Z.18.2-95,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [72]:
cases_reshape = cases_reshape.assign(total=cases_reshape['American Indian or Alaskan Native'] + 
                                              cases_reshape['Asian or Pacific Islander'] +
                                              cases_reshape['Black'] + 
                                              cases_reshape['Hispanic'] + 
                                              cases_reshape['Missing/Other'] + 
                                              cases_reshape['White'])

In [73]:
cases_reshape = cases_reshape.query("total > 50")

In [74]:
cases_reshape = cases_reshape.assign(black_percent = cases_reshape['Black']/cases_reshape['total'],
                                     white_percent = cases_reshape['White']/cases_reshape['total'])


In [75]:
cases_reshape.sort_values('black_percent', ascending=False)

Race,CodeSection,,American Indian or Alaskan Native,Asian or Pacific Islander,Black,Hispanic,Missing/Other,White,total,black_percent,white_percent
1225,24-253,0.0,0.0,0.0,75.0,0.0,0.0,3.0,78.0,0.961538,0.038462
1381,29-48,0.0,0.0,0.0,127.0,0.0,1.0,11.0,139.0,0.913669,0.079137
2022,46.2-938,0.0,0.0,1.0,180.0,2.0,2.0,36.0,221.0,0.814480,0.162896
2384,66-3,0.0,0.0,0.0,65.0,0.0,0.0,18.0,83.0,0.783133,0.216867
814,18.2-53.1,0.0,0.0,2.0,466.0,13.0,1.0,137.0,619.0,0.752827,0.221325
...,...,...,...,...,...,...,...,...,...,...,...
710,18.2-374.1:1,0.0,0.0,0.0,46.0,1.0,0.0,375.0,422.0,0.109005,0.888626
1408,29.1-335,0.0,0.0,3.0,23.0,11.0,3.0,184.0,224.0,0.102679,0.821429
2260,54.1-3466,0.0,0.0,1.0,28.0,1.0,1.0,245.0,276.0,0.101449,0.887681
1423,29.1-521,0.0,0.0,0.0,7.0,0.0,0.0,109.0,116.0,0.060345,0.939655


### In what localities (fips) are these disparities most severe? 

In [ ]:
cases